In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import evalml
import time

In [2]:
df = pd.read_csv('wustl_hdrl_2024.csv')
df.head()

C:\Users\nakul\AppData\Local\Temp\ipykernel_5844\3850691082.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('wustl_hdrl_2024.csv')


,Dir,Flgs,SrcAddr,DstAddr,Sport,Dport,SrcBytes,DstBytes,SrcLoad,DstLoad,...,sswap_sin,sswap_sout,sdiskusage_total,sdiskusage_used,sdiskusage_free,sdiskusage_percent,Boot_Time_with_date,DTime,Attack_categories,Label
0,->,e,192.168.3.2,192.168.2.2,36390,4022,1651.0,272.0,10440.0,1612.000,...,0,0,502392610816,51814645760,424982593536,10,2023-12-23 15:38:11,104.316401,normal,0
1,->,e,192.168.3.2,192.168.2.2,49608,4022,1673.0,272.0,10576.0,1611.000,...,0,0,502392610816,51814674432,424982564864,10,2023-12-23 15:38:11,104.207156,normal,0
2,->,e,192.168.3.2,192.168.2.2,34596,4022,1650.0,272.0,10364.0,1602.000,...,0,0,502392610816,51814686720,424982552576,10,2023-12-23 15:38:11,104.177347,normal,0
3,->,e d,78.151.208.250,192.168.2.2,18346,4022,1450.0,174.0,0.0,306.469,...,0,0,502392610858,50459127848,426338111537,15,2023-05-09 15:19:38,25.821966,DDoS,1
4,->,e d,143.109.155.233,192.168.2.2,18346,4022,1452.0,174.0,0.0,305.268,...,0,0,502392610863,50459127854,426338111539,15,2023-05-09 15:19:38,25.945508,DDoS,1


In [3]:
df['Attack_categories'].value_counts()

Attack_categories
normal             132884
DDoS                 9971
MiTM                 1672
Ransomware            528
Buffer_overflow        68
Name: count, dtype: int64

In [4]:
df.shape

(145123, 77)

In [5]:
df=df.sample(frac=1)
df.reset_index(drop=True,inplace=True)

In [6]:
df = df.loc[:, df.nunique() > 1]
df.shape

(145123, 64)

In [7]:
df.head(10)

,Dir,Flgs,SrcAddr,DstAddr,Sport,SrcBytes,DstBytes,SrcLoad,DstLoad,SrcGap,...,sswap_total,sswap_free,sdiskusage_total,sdiskusage_used,sdiskusage_free,sdiskusage_percent,Boot_Time_with_date,DTime,Attack_categories,Label
0,->,e,192.168.3.2,192.168.2.2,59904,1664.0,272.0,10519.0,1611.000,0.0,...,2147479552,2147479552,502392610816,51788685312,425008553984,10,2023-12-23 15:38:11,105.965326,normal,0
1,->,e,192.168.3.2,192.168.2.2,60056,1686.0,272.0,10655.0,1611.000,0.0,...,2147479552,2147479552,502392610816,51785428992,425011810304,10,2023-12-23 15:38:11,105.693339,normal,0
2,->,e,198.51.100.100,192.168.4.2,33944,1658.0,272.0,10459.0,1608.000,0.0,...,104853504,104853504,30612852736,17902596096,11378843648,61,2023-12-23 15:40:42,24.761527,normal,0
3,->,e,198.51.100.100,192.168.4.2,57634,1655.0,272.0,10435.0,1608.000,0.0,...,104853504,104853504,30612852736,16820568064,12460871680,57,2023-12-23 15:40:42,23.557805,normal,0
4,->,e d,61.131.28.49,192.168.2.2,34463,1454.0,174.0,0.0,306.044,0.0,...,2147480354,2147480392,502392611609,50459128746,426338112360,10,2023-05-09 15:19:38,28.619525,DDoS,1
5,->,e,198.51.100.100,192.168.4.2,58360,1652.0,272.0,10421.0,1608.000,0.0,...,104853504,104853504,30612852736,15914336256,13367103488,54,2023-12-23 15:40:42,22.403976,normal,0
6,->,e,198.51.100.100,192.168.4.2,39322,1648.0,272.0,10398.0,1608.000,0.0,...,104853504,104853504,30612852736,19173957632,10107482112,65,2023-12-23 15:40:42,26.973776,normal,0
7,->,e,192.168.6.2,192.168.4.2,40670,1949.0,272.0,13568.0,1597.000,0.0,...,2147479552,2147479552,502392610816,15318261760,461478977536,3,2023-12-23 15:40:19,9.569550,normal,0
8,->,e,192.168.3.2,192.168.2.2,60188,1675.0,272.0,10583.0,1611.000,0.0,...,2147479552,2147479552,502392610816,51797233664,425000005632,10,2023-12-23 15:38:11,105.082596,normal,0
9,->,e,198.51.100.100,192.168.4.2,49614,1657.0,272.0,10448.0,1607.000,0.0,...,104853504,104853504,30612852736,19727507456,9553932288,67,2023-12-23 15:40:42,27.358845,normal,0


In [8]:
X = df.drop(['Attack_categories','Label'],axis=1)
y = df['Label']
y.value_counts()

Label
0    132884
1     12239
Name: count, dtype: int64

In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145123 entries, 0 to 145122
Data columns (total 62 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Dir                        145123 non-null  object 
 1   Flgs                       145123 non-null  object 
 2   SrcAddr                    145123 non-null  object 
 3   DstAddr                    145123 non-null  object 
 4   Sport                      145123 non-null  object 
 5   SrcBytes                   145123 non-null  float64
 6   DstBytes                   145123 non-null  float64
 7   SrcLoad                    145123 non-null  float64
 8   DstLoad                    145123 non-null  float64
 9   SrcGap                     145123 non-null  float64
 10  SIntPkt                    145123 non-null  float64
 11  DIntPkt                    145123 non-null  float64
 12  SIntPktAct                 145123 non-null  float64
 13  DIntPktAct                 14

In [10]:
non_numeric_features = X.select_dtypes(exclude=['number']).columns.tolist()
non_numeric_features

['Dir',
 'Flgs',
 'SrcAddr',
 'DstAddr',
 'Sport',
 'SrcMac',
 'DstMac',
 'RTime',
 'Boot_Time_with_date']

In [11]:
X[non_numeric_features]

,Dir,Flgs,SrcAddr,DstAddr,Sport,SrcMac,DstMac,RTime,Boot_Time_with_date
0,->,e,192.168.3.2,192.168.2.2,59904,10:62:e5:0a:b8:c0,9c:5a:44:39:14:be,2024-01-06 02:45:28.098340,2023-12-23 15:38:11
1,->,e,192.168.3.2,192.168.2.2,60056,10:62:e5:0a:b8:c0,9c:5a:44:39:14:be,2024-01-02 14:48:20.556271,2023-12-23 15:38:11
2,->,e,198.51.100.100,192.168.4.2,33944,00:10:18:a1:34:64,00:68:eb:d5:65:3b,2024-01-01 23:02:19.845995,2023-12-23 15:40:42
3,->,e,198.51.100.100,192.168.4.2,57634,00:10:18:a1:34:64,00:68:eb:d5:65:3b,2023-12-28 22:14:01.631671,2023-12-23 15:40:42
4,->,e d,61.131.28.49,192.168.2.2,34463,10:62:e5:0a:b8:c0,9c:5a:44:39:14:be,2024-01-05 03:22:28.734849,2023-05-09 15:19:38
...,...,...,...,...,...,...,...,...,...
145118,->,e,192.168.3.2,192.168.2.2,57504,10:62:e5:0a:b8:c0,9c:5a:44:39:14:be,2024-01-02 08:18:40.296781,2023-12-23 15:38:11
145119,->,e d,83.64.175.82,192.168.4.2,50831,00:10:18:a1:34:64,00:68:eb:d5:65:3b,2024-01-02 19:21:32.170872,2023-05-09 15:19:38
145120,->,e,192.168.3.2,192.168.2.2,47630,10:62:e5:0a:b8:c0,9c:5a:44:39:14:be,2024-01-01 03:57:22.532038,2023-12-23 15:38:11
145121,->,e,198.51.100.100,192.168.4.2,39498,00:10:18:a1:34:64,00:68:eb:d5:65:3b,2024-01-02 15:28:03.243896,2023-12-23 15:40:42


In [12]:
X.drop(['RTime','Boot_Time_with_date','Dir'],axis=1,inplace=True)

In [13]:
le = LabelEncoder()
X['Sport']=X['Sport'].astype('str')
X['Sport'] = le.fit_transform(X['Sport'])
X['SrcMac'] = le.fit_transform(X['SrcMac'])
X['DstMac'] = le.fit_transform(X['DstMac'])
X['DstAddr'] = le.fit_transform(X['DstAddr'])

In [14]:
X.drop(['Flgs','SrcAddr','IMEI','sMinPktSz'],inplace=True,axis=1)
X.shape

(145123, 55)

In [15]:
y.value_counts()

Label
0    132884
1     12239
Name: count, dtype: int64

In [16]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

In [17]:
X_train,X_test,y_train,y_test = train_test_split(np.array(X),y,test_size=0.2,stratify = y)

In [21]:
import time
import tracemalloc
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_absolute_error
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Assuming X_train, X_test, y_train, y_test are already defined

# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Perceptron": Perceptron(),
    "Simple MLP": MLPClassifier(max_iter=1000),
    "SVM": SVC(probability=True),
    "KNN": KNeighborsClassifier(),
    "SGDClassifier": SGDClassifier()
}

# Initialize results dictionary
results = {
    "Model": [],
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1-Score": [],
    "ROC AUC Score": [],
    "MAE": [],
    "Training Time (s)": [],
    "Memory Used (MB)": []
}

# Train and evaluate each model
for model_name, model in models.items():
    print(f"Training {model_name}...")
    
    # Start tracking memory and time
    tracemalloc.start()
    start_time = time.time()
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
    
    # Stop tracking memory and time
    training_time = time.time() - start_time
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='weighted')
    roc_auc = roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else np.nan
    mae = mean_absolute_error(y_test, y_pred)
    
    # Append results
    results["Model"].append(model_name)
    results["Accuracy"].append(accuracy)
    results["Precision"].append(precision)
    results["Recall"].append(recall)
    results["F1-Score"].append(f1)
    results["ROC AUC Score"].append(roc_auc)
    results["MAE"].append(mae)
    results["Training Time (s)"].append(training_time)
    results["Memory Used (MB)"].append(peak / 10**6)  # Convert to MB


Training Logistic Regression...
Training Random Forest...
Training Decision Tree...
Training Perceptron...
Training Simple MLP...
Training SVM...
Training KNN...
Training SGDClassifier...


In [22]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)
print(results_df)

                 Model  Accuracy  Precision    Recall  F1-Score  \
0  Logistic Regression  0.985185   0.984973  0.926452  0.984781   
1        Random Forest  0.999414   0.999414  0.997640  0.999414   
2        Decision Tree  0.998553   0.998554  0.995687  0.998554   
3           Perceptron  0.975263   0.977231  0.949405  0.975943   
4           Simple MLP  0.996865   0.996861  0.989202  0.996863   
5                  SVM  0.988424   0.988424  0.936008  0.988096   
6                  KNN  0.995935   0.995916  0.982945  0.995919   
7        SGDClassifier  0.985702   0.985905  0.915608  0.985102   

   ROC AUC Score       MAE  Training Time (s)  Memory Used (MB)  
0       0.991821  0.014815           4.069971         23.777835  
1       0.999991  0.000586          62.435559         34.631317  
2       0.995687  0.001447           7.750707         31.301644  
3            NaN  0.024737           0.350016          2.564115  
4       0.999553  0.003135         164.842927         23.833026  


In [23]:
from sklearn.ensemble import HistGradientBoostingClassifier
import time
base_clf = HistGradientBoostingClassifier()
stime=time.time()
base_clf.fit(X_train,y_train)
print('Time is:',time.time()-stime)

Time is: 2.7247211933135986
